In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline

# Data Setup & Functions

In [3]:
from mll_calc.mll_pred import format_XY

In [4]:
def logpdf_calc(row, test_sample, unc):
    y_sim = row[test_sample>0].values.tolist()
    std = row.multiply(unc)[test_sample>0].values.tolist()
    y_mes = test_sample[test_sample>0].values.tolist()
    logpdf_list = stats.norm.logpdf(y_sim, loc=y_mes, scale=std)
    return logpdf_list

In [7]:
def loop_db(XY, pred, unc, lbls, nonlbls):
    #xy_cols = XY.columns.tolist()
    #for col in nonlbls: xy_cols.remove(col)
    #test = test[xy_cols]
    
    logpdf_df = pd.DataFrame()
    for idx, row in pred.iterrows():
        sim_idx = row['sim_idx']
        pred_idx = row['pred_idx']
        all_lbls = lbls + nonlbls
        
        test_sample = XY.loc[sim_idx].drop(all_lbls)
        train_row = XY.loc[pred_idx].drop(all_lbls)
        
        logpdf = logpdf_calc(train_row, test_sample, unc)
        mll = row['MaxLogLL']
        
        if round(mll, 4) != round(np.sum(logpdf), 4):
            print('mismatch')
            break
        
        logpdf = pd.Series(logpdf, index=train_row[test_sample>0].index, name=sim_idx)
        if logpdf_df.empty:
            logpdf_df = pd.DataFrame(columns = test_sample.index.to_list())
        logpdf_df = logpdf_df.append(logpdf)
    return logpdf_df

### Train and Test DBs

In [8]:
lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
nonlbls = ['AvgPowerDensity', 'ModDensity', 'UiWeight']

train_pkl = '~/sims_n_results/simupdates_aug2020/not-scaled_nuc29.pkl'
XY = format_XY(train_pkl)

### LL Calc Results

In [9]:
sfcompo_results = '~/sims_n_results/simupdates_aug2020/train29/'

uncs = [0.05, 0.1, 0.15, 0.2]
job_dirs = ['Job' + str(i) + '_unc' + str(unc) for i, unc in enumerate(uncs)]
dfs = {}
for i, unc_job in enumerate(job_dirs):
    dfs['unc' + str(i)] = pd.read_csv(sfcompo_results + unc_job + '/' + unc_job + '.csv')

for dfname in list(dfs.keys()):
    df = dfs[dfname]
    df['Relative_Burnup_Error'] = df['Burnup_Error'] / df['Burnup']
    df['Relative_Enrichment_Error'] = df['Enrichment_Error'] / df['Enrichment']
    df['Relative_CoolingTime_Error'] = df['CoolingTime_Error'] / df['CoolingTime']

# For ease of only viewing 5% case for now
preds = dfs['unc0']

### LogPDF (wrt nuclide) results

In [10]:
unc = 0.05
logpdf_df = loop_db(XY, preds, unc, lbls, nonlbls)

KeyboardInterrupt: 

# Explore Errors

### Burnup

In [11]:
preds['Relative_Burnup_Error'].describe()
# nuc15 results for comparison
#count    9600.000000
#mean        0.002187
#std         0.006690
#min         0.000000
#25%         0.000000
#50%         0.000000
#75%         0.000000
#max         0.064146

count    12000.000000
mean         0.003525
std          0.009948
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          0.149049
Name: Relative_Burnup_Error, dtype: float64

In [14]:
to_print = ['sim_idx', 'pred_idx', 'AvgPowerDensity', 
            'ReactorType', 'pred_ReactorType', 'ReactorType_Score', 
            'Enrichment', 'pred_Enrichment', 'Relative_Enrichment_Error', 
            'Burnup', 'pred_Burnup', 'Relative_Burnup_Error',
            'CoolingTime', 'pred_CoolingTime', 'Relative_CoolingTime_Error',
            'OrigenReactor', 'pred_OrigenReactor', 'MaxLogLL']
preds.loc[preds['Relative_Burnup_Error'] > 0.08, to_print]

,sim_idx,pred_idx,AvgPowerDensity,ReactorType,pred_ReactorType,ReactorType_Score,Enrichment,pred_Enrichment,Relative_Enrichment_Error,Burnup,pred_Burnup,Relative_Burnup_Error,CoolingTime,pred_CoolingTime,Relative_CoolingTime_Error,OrigenReactor,pred_OrigenReactor,MaxLogLL
368,11299,147378,10.0,bwr,bwr,True,1.570,1.870,0.191083,36458.87,39461.41,0.082354,1520.464582,1502.271013,0.011966,ge7x7-0,atrium10x10-9,-122.232230
999,30560,91099,10.0,bwr,bwr,True,2.740,3.100,0.131387,7974.93,8683.04,0.088792,1706.758096,1708.649694,0.001108,ge7x7-0,abb8x8-1,-69.409535
2309,70563,136082,10.0,bwr,bwr,True,1.380,1.610,0.166667,944.34,1030.70,0.091450,7.797336,1.002610,0.871416,abb8x8-1,atrium10x10-9,9.789710
2807,85858,20338,10.0,bwr,bwr,True,2.080,2.030,0.024038,3250.48,2970.68,0.086080,7120.206752,6973.335475,0.020627,abb8x8-1,ge7x7-0,-21.771372
3298,100926,151326,10.0,bwr,bwr,True,4.000,3.280,0.180000,3250.48,2766.00,0.149049,210.929088,242.633977,0.150311,abb8x8-1,atrium10x10-9,-105.995935
5279,161525,45605,10.0,bwr,bwr,True,4.230,4.400,0.040189,5670.60,6261.29,0.104167,109.336506,84.338574,0.228633,atrium10x10-9,ge7x7-0,-62.594131
5293,161885,45965,10.0,bwr,bwr,True,4.230,4.400,0.040189,14697.28,16053.83,0.092299,109.336506,84.338574,0.228633,atrium10x10-9,ge7x7-0,-101.280906
7081,216785,95825,10.0,bwr,bwr,True,2.730,3.100,0.135531,1909.14,2107.24,0.103764,101.346301,98.925867,0.023883,svea64-1,abb8x8-1,-38.692282
9275,283935,173957,25.0,pwr,bwr,False,4.520,5.110,0.130531,20178.59,21847.13,0.082689,1270.348072,1298.076612,0.021828,w17x17,atrium10x10-9,-119.307797
10760,329396,261416,25.0,pwr,pwr,True,4.990,5.390,0.080160,25193.99,27430.12,0.088756,6262.694626,6131.154219,0.021004,bw15x15,ce14x14,-108.454521


### Enrichment

In [15]:
preds['Relative_Enrichment_Error'].describe()
# nuc15 results for comparison
#count    9600.000000
#mean        0.004361
#std         0.019275
#min         0.000000
#25%         0.000000
#50%         0.000000
#75%         0.000000
#max         0.303448

count    12000.000000
mean         0.003752
std          0.014869
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          0.248447
Name: Relative_Enrichment_Error, dtype: float64

In [19]:
preds.loc[preds['Relative_Enrichment_Error'] > 0.12, to_print]

,sim_idx,pred_idx,AvgPowerDensity,ReactorType,pred_ReactorType,ReactorType_Score,Enrichment,pred_Enrichment,Relative_Enrichment_Error,Burnup,pred_Burnup,Relative_Burnup_Error,CoolingTime,pred_CoolingTime,Relative_CoolingTime_Error,OrigenReactor,pred_OrigenReactor,MaxLogLL
368,11299,147378,10.0,bwr,bwr,True,1.57,1.87,0.191083,36458.87,39461.41,0.082354,1520.464582,1502.271013,0.011966,ge7x7-0,atrium10x10-9,-122.232230
531,16338,71656,10.0,bwr,bwr,True,1.57,1.38,0.121019,36458.87,34658.13,0.049391,1385.096982,1340.825727,0.031963,ge7x7-0,abb8x8-1,-119.813632
999,30560,91099,10.0,bwr,bwr,True,2.74,3.10,0.131387,7974.93,8683.04,0.088792,1706.758096,1708.649694,0.001108,ge7x7-0,abb8x8-1,-69.409535
2306,70566,10086,10.0,bwr,bwr,True,1.38,1.57,0.137681,944.34,965.63,0.022545,210.929088,208.534543,0.011352,abb8x8-1,ge7x7-0,-8.041813
2309,70563,136082,10.0,bwr,bwr,True,1.38,1.61,0.166667,944.34,1030.70,0.091450,7.797336,1.002610,0.871416,abb8x8-1,atrium10x10-9,9.789710
3298,100926,151326,10.0,bwr,bwr,True,4.00,3.28,0.180000,3250.48,2766.00,0.149049,210.929088,242.633977,0.150311,abb8x8-1,atrium10x10-9,-105.995935
4280,131105,70625,10.0,bwr,bwr,True,1.61,1.38,0.142857,2132.20,2107.24,0.011706,109.336506,98.925867,0.095216,atrium10x10-9,abb8x8-1,-34.274453
4320,132306,147426,10.0,bwr,bwr,True,1.61,1.87,0.161491,43096.56,43096.56,0.000000,242.633977,242.633977,0.000000,atrium10x10-9,atrium10x10-9,-137.796529
4324,132318,207976,10.0,bwr,bwr,True,1.61,2.01,0.248447,43096.56,45382.93,0.053052,1502.271013,1495.612006,0.004433,atrium10x10-9,svea64-1,-130.158957
4625,141555,131415,10.0,bwr,bwr,True,1.87,1.61,0.139037,9700.27,9304.01,0.040850,1053.708101,1053.708101,0.000000,atrium10x10-9,atrium10x10-9,-87.300122


### Cooling Time

In [20]:
preds['Relative_CoolingTime_Error'].describe()
# nuc15 results for comparison
#count    9600.000000
#mean             inf
#std              NaN
#min         0.000000
#25%         0.004190
#50%         0.013621
#75%         0.047517
#max              inf

count    1.200000e+04
mean              inf
std               NaN
min      0.000000e+00
25%      3.333665e-03
50%      1.209631e-02
75%      2.999413e-02
max               inf
Name: Relative_CoolingTime_Error, dtype: float64

In [26]:
preds.loc[(preds['Relative_CoolingTime_Error'] != np.inf) & (preds['Relative_CoolingTime_Error'] > 2.9), to_print]

,sim_idx,pred_idx,AvgPowerDensity,ReactorType,pred_ReactorType,ReactorType_Score,Enrichment,pred_Enrichment,Relative_Enrichment_Error,Burnup,pred_Burnup,Relative_Burnup_Error,CoolingTime,pred_CoolingTime,Relative_CoolingTime_Error,OrigenReactor,pred_OrigenReactor,MaxLogLL
7,303,304,10.0,bwr,bwr,True,0.500,0.500,0.0,7974.93,7974.93,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-64.515327
108,3243,3244,20.0,bwr,bwr,True,0.500,0.500,0.0,63534.57,63534.57,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-108.271418
140,4323,4324,25.0,bwr,bwr,True,0.500,0.500,0.0,29424.92,29424.92,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-98.302174
556,16983,16984,20.0,bwr,bwr,True,1.570,1.570,0.0,4769.54,4769.54,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-45.095398
604,18424,18425,20.0,bwr,bwr,True,1.570,1.570,0.0,70500.00,70500.00,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-112.081377
633,19383,19384,25.0,bwr,bwr,True,1.570,1.570,0.0,22690.61,22690.61,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-93.441531
650,19923,19924,25.0,bwr,bwr,True,1.570,1.570,0.0,51880.09,51880.09,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-109.741057
687,21063,21064,10.0,bwr,bwr,True,2.030,2.030,0.0,22690.61,22690.61,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-94.060324
690,21123,21124,10.0,bwr,bwr,True,2.030,2.030,0.0,29424.92,29424.92,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-101.480483
754,23043,23044,20.0,bwr,bwr,True,2.030,2.030,0.0,36458.87,36458.87,0.0,6.392266,25.487227,2.987197,ge7x7-0,ge7x7-0,-106.260106


# Max Log LL

In [41]:
print(dfs['unc0']['MaxLogLL'].describe())
print(dfs['unc3']['MaxLogLL'].describe())

count    9600.000000
mean      -38.294298
std        19.291833
min       -65.162488
25%       -52.718086
50%       -43.620235
75%       -29.973796
max        38.627426
Name: MaxLogLL, dtype: float64
count    9600.000000
mean      -58.051921
std        19.337723
min       -84.003339
25%       -72.697537
50%       -63.698441
75%       -49.036952
max        17.588901
Name: MaxLogLL, dtype: float64


# Nuclide Contributions to LL

In [23]:
nuc29 = ['am241', 'am242m', 'am243', 
         'cm242', 'cm244', 
         'cs134', 'cs137', 
         'eu154', 
         'nd143', 'nd144', 'nd145', 'nd146', 'nd148', 'nd150',
         'np237', 
         'pu238', 'pu239', 'pu240', 'pu241', 'pu242',
         'sm147', 'sm149', 'sm150', 'sm151', 'sm152',
         'u234', 'u235', 'u236', 'u238']

In [24]:
logpdf_df.sample(10)

,ba136,ba138,cs133,cs134,cs135,cs137,eu153,eu154,pu239,pu240,pu241,pu242,sm149,sm150,sm152
176653,2.319375,-3.387038,-3.339751,1.407995,-2.927823,-3.229284,-0.082152,2.899087,-5.455830,-2.840263,-1.186824,2.138776,0.943136,-1.620798,-0.884347
355828,-0.410610,-4.802253,-4.707098,-0.577727,-3.200811,-4.620977,-2.498433,-0.355834,-6.383617,-5.444583,-4.809237,-4.390750,1.053983,-3.373815,-2.464904
270419,-0.898736,-5.119143,-4.962867,-3.026725,-3.433808,-5.093326,-2.846830,-1.303180,-6.327231,-5.650741,-5.171710,-4.934420,1.707993,-3.713778,-2.670744
356505,-1.919203,-5.627567,-5.356282,-0.090477,-4.217919,-5.301289,-3.282619,-0.894618,-6.508432,-5.951094,-4.840612,-5.469653,0.946037,-4.189268,-2.923791
17563,0.148757,-4.463226,-4.397560,2.166844,-2.983266,-4.144875,-1.986404,0.845718,-6.002514,-5.209398,-3.918424,-3.656548,1.577838,-2.945770,-2.262325
80354,-1.950893,-5.556063,-5.279213,-2.412255,-4.053660,-5.431140,-3.137696,-1.182953,-5.999720,-5.721846,-4.735958,-5.511532,1.436709,-3.967483,-3.004538
179998,2.925267,-3.213312,-3.162227,6.930995,-2.273192,-2.624470,0.133952,4.785102,-5.354354,-2.414892,0.306936,2.655562,0.696148,-1.402410,-0.883510
162872,-2.685380,-5.823843,-5.533564,-0.442411,-5.214862,-5.375555,-3.320916,-1.087142,-6.422846,-5.642820,-4.868804,-5.269861,1.465399,-4.137414,-2.956907
286034,4.240487,-2.287832,-2.247736,3.965999,-1.117789,-2.134546,1.174602,5.028589,-4.690664,-1.177479,1.373170,5.758499,0.866938,-0.316546,0.470561
363586,0.744956,-3.959644,-3.927340,3.459895,-1.770157,-3.635948,-1.430864,1.559787,-5.816778,-5.153769,-3.254475,-3.098225,1.869780,-2.440201,-1.973076


In [26]:
for nuc in nuc15:
    desc = logpdf_df[nuc].describe()
    print(desc[['mean', 'std', 'min', 'max']])

mean   -4.405521
std     0.965536
min    -6.029460
max    -0.748743
Name: cs133, dtype: float64
mean     0.553483
std      2.621435
min     -8.562148
max     10.851586
Name: cs134, dtype: float64
mean   -3.342744
std     1.198377
min    -8.581306
max     1.405381
Name: cs135, dtype: float64
mean   -4.276594
std     1.035930
min    -5.794990
max    -0.277484
Name: cs137, dtype: float64
mean   -1.885695
std     1.335974
min    -3.630491
max     2.702843
Name: eu153, dtype: float64
mean    0.674270
std     1.913928
min    -2.123420
max     8.071268
Name: eu154, dtype: float64
mean    0.016693
std     1.819714
min    -3.955707
max     5.819496
Name: ba136, dtype: float64
mean   -4.524608
std     1.035798
min    -6.281010
max    -0.781399
Name: ba138, dtype: float64
mean    1.254416
std     0.479296
min    -4.785105
max     3.130911
Name: sm149, dtype: float64
mean   -2.937306
std     1.119736
min    -4.654297
max     1.069201
Name: sm150, dtype: float64
mean   -2.100939
std     0.974940
mi